In [5]:
!pip install -q -U google-genai

## Entity Extraction into JSON/Lists

In [ ]:
from pydantic import BaseModel

class Recipe(BaseModel):
  recipe_name: str
  ingredients: list[str]

Recipe(recipe_name="Pasta", ingredients=["flour", "water"])
# Recipe(recipe_name=1, ingredients=["flour", "water"])

Recipe(recipe_name='hi', ingredients=['flour', 'water'])

In [34]:
from google import genai
# import google.generativeai as genai
from pydantic import BaseModel
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Configure the API key
# genai.configure(api_key=os.environ["GEMINI_API_KEY"])

class Recipe(BaseModel):
  recipe_name: str
  ingredients: list[str]


client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few popular cookie recipes.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Recipe],
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

[
  {
    "recipe_name": "Chocolate Chip Cookies",
    "ingredients": [
      "Butter",
      "Sugar",
      "Brown Sugar",
      "Eggs",
      "Vanilla Extract",
      "Flour",
      "Baking Soda",
      "Salt",
      "Chocolate Chips"
    ]
  },
  {
    "recipe_name": "Peanut Butter Cookies",
    "ingredients": [
      "Peanut Butter",
      "Butter",
      "Sugar",
      "Brown Sugar",
      "Egg",
      "Vanilla Extract",
      "Flour",
      "Baking Soda",
      "Salt"
    ]
  },
  {
    "recipe_name": "Oatmeal Raisin Cookies",
    "ingredients": [
      "Butter",
      "Brown Sugar",
      "Egg",
      "Vanilla Extract",
      "Flour",
      "Baking Soda",
      "Salt",
      "Cinnamon",
      "Oats",
      "Raisins"
    ]
  }
]


In [ ]:
# Exercise extract the list and run for loop through it.

In [15]:
class Thaifood(BaseModel):
    Location: str
    food: list[str]

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few popular food in Thailand based on location.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Thaifood],
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

[
  {
    "Location": "Bangkok",
    "food": ["Pad Thai", "Mango Sticky Rice", "Tom Yum Goong", "Green Curry"]
  },
  {
    "Location": "Chiang Mai",
    "food": ["Khao Soi", "Sai Oua (Northern Thai Sausage)", "Gaeng Hang Lay (Burmese-style Pork Curry)"]
  },
  {
    "Location": "Southern Thailand (e.g., Phuket, Krabi)",
    "food": ["Massaman Curry", "Roti", "Seafood (grilled fish, prawns)", "Tom Yum Pla (Fish Tom Yum)"]
  },
  {
    "Location": "Isan (Northeast Thailand)",
    "food": ["Som Tum (Papaya Salad)", "Larb (Minced Meat Salad)", "Gai Yang (Grilled Chicken)", "Sticky Rice"]
  }
]


In [ ]:
# Exercise extract the list and run for loop through it.
# Exercise ask the model list cities in Thailand, make model return a list of city. Run For loop through it.

In [20]:
class ThaifoodBangkok(BaseModel):
    food: list[str]

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List a few popular food in Bangkok.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': ThaifoodBangkok,
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

{
  "food": ["Pad Thai", "Mango Sticky Rice", "Tom Yum Soup", "Green Curry", "Boat Noodles"]
}


In [28]:
for food in response.parsed.food:
    print(food)

Pad Thai
Mango Sticky Rice
Tom Yum Soup
Green Curry
Boat Noodles


## Classification

In [31]:
import enum

class Instrument(enum.Enum):
  PERCUSSION = "Percussion"
  STRING = "String"
  WOODWIND = "Woodwind"
  BRASS = "Brass"
  KEYBOARD = "Keyboard"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='What type of instrument is an oboe?',
    config={
        'response_mime_type': 'text/x.enum',
        'response_schema': Instrument,
    },
)

print(response.text)

Woodwind


In [32]:
class Grade(enum.Enum):
    A_PLUS = "a+"
    A = "a"
    B = "b"
    C = "c"
    D = "d"
    F = "f"

class Recipe(BaseModel):
  recipe_name: str
  rating: Grade

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='List 10 home-baked cookies and give them grades based on tastiness.',
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Recipe],
    },
)

print(response.text)

[
  {
    "recipe_name": "Chocolate Chip Cookies",
    "rating": "a+"
  },
  {
    "recipe_name": "Peanut Butter Cookies",
    "rating": "a"
  },
  {
    "recipe_name": "Oatmeal Raisin Cookies",
    "rating": "b"
  },
  {
    "recipe_name": "Sugar Cookies",
    "rating": "b"
  },
  {
    "recipe_name": "Snickerdoodles",
    "rating": "a"
  },
  {
    "recipe_name": "Gingerbread Cookies",
    "rating": "c"
  },
  {
    "recipe_name": "Macadamia Nut Cookies",
    "rating": "a"
  },
  {
    "recipe_name": "Double Chocolate Cookies",
    "rating": "a+"
  },
  {
    "recipe_name": "Shortbread Cookies",
    "rating": "b"
  },
  {
    "recipe_name": "Lemon Cookies",
    "rating": "b"
  }
]


In [ ]:
# Get a newspaper extract date, summary, title in Json format each with strings.